# importing data

In [4]:
"""
Simple tester for the vgg19_trainable
"""

import tensorflow as tf
import Deep3D as deep3d
import utils

import os
import os.path
import h5py

In [7]:
inria_file = 'inria_data.h5'
h5f = h5py.File(inria_file,'r')

X_train = h5f['X_0'][:400]
Y_train = h5f['Y_0'][:400]
X_val = h5f['X_0'][400:]
Y_val = h5f['Y_0'][400:]
   
    
h5f.close()

In [8]:
print Y_train.shape

(400, 180, 320, 3)


In [ ]:
num_batches = 100
batchsize = 50

with tf.device('/cpu:0'):
    sess = tf.Session()

    images = tf.placeholder(tf.float32, [-1, 180, 320, 3])
    
    true_out = tf.placeholder(tf.float32, [-1, 180, 320, 3])
    train_mode = tf.placeholder(tf.bool)

    net = deep3d.Deep3Dnet('./deep3d-save.npy')
    net.build(images, train_mode)

    # print number of variables used: 143667240 variables, i.e. ideal size = 548MB
    print(net.get_var_count())

    sess.run(tf.global_variables_initializer())

    # test classification
    prob = sess.run(net.prob, feed_dict={images: batch1, train_mode: False})

    # simple 1-step training
    cost = tf.reduce_sum((vgg.prob - true_out) ** 2)
    train = tf.train.GradientDescentOptimizer(0.0001).minimize(cost)
    for i in xrange(num_batches):
        batch1 = np.random(X_train,batchsize)
        sess.run(train, feed_dict={images: batch1, true_out: [img1_true_result], train_mode: True})

    # test classification again, should have a higher probability about tiger
    prob = sess.run(vgg.prob, feed_dict={images: batch1, train_mode: False})
    utils.print_prob(prob[0], './synset.txt')

    # test save
    #vgg.save_npy(sess, './test-save.npy')

ValueError: Dimension -1 must be >= 0

# testing the network

In [ ]:
img1 = utils.load_image("./test_data/tiger.jpeg")
img1_true_result = [1 if i == 292 else 0 for i in range(1000)]  # 1-hot result for tiger

batch1 = img1.reshape((1, 180, 320, 3))

with tf.device('/gpu:0'):
    sess = tf.Session()

    images = tf.placeholder(tf.float32, [1, 180, 320, 3])
    true_out = tf.placeholder(tf.float32, [1, 1000])
    train_mode = tf.placeholder(tf.bool)

    vgg = deep3d.Deep3Dnet('./deep3d-save.npy')
    vgg.build(images, train_mode)

    # print number of variables used: 143667240 variables, i.e. ideal size = 548MB
    print(vgg.get_var_count())

    sess.run(tf.global_variables_initializer())

    # test classification
    prob = sess.run(vgg.prob, feed_dict={images: batch1, train_mode: False})
    utils.print_prob(prob[0], './synset.txt')

    # simple 1-step training
    cost = tf.reduce_sum((vgg.prob - true_out) ** 2)
    train = tf.train.GradientDescentOptimizer(0.0001).minimize(cost)
    sess.run(train, feed_dict={images: batch1, true_out: [img1_true_result], train_mode: True})

    # test classification again, should have a higher probability about tiger
    prob = sess.run(vgg.prob, feed_dict={images: batch1, train_mode: False})
    utils.print_prob(prob[0], './synset.txt')

    # test save
    vgg.save_npy(sess, './test-save.npy')